In [1]:
import os
from omegaconf import DictConfig

In [2]:
files_list: list[dict[str, str | list]] = []
files_list

[]

In [3]:
configs = [
    ['/home/sha/work/yi/data-scripts-queries/ecs', '/home/sha/work/yi/data-scripts-queries/queries/query_ecs_yi.txt', 'ecs'],
    ['/home/sha/work/yi/data-scripts-queries/pwy', '/home/sha/work/yi/data-scripts-queries/queries/query_pwy_yi.txt', 'pwy'],
    ['/home/sha/work/yi/data-scripts-queries/tax', '/home/sha/work/yi/data-scripts-queries/queries/query_tax_yi.txt', 'tax'],
]

In [4]:
for item in configs:
    if item[0] is not None and item[1] is not None:
            files_list.append({"name": item[2], "path": item[2], "files": []})
    else:
        pass

In [5]:
# files_list

In [6]:
queries: dict[str, str] = {}
with open('/home/sha/work/yi/data-scripts-queries/queries/query_meta.txt', "r") as file:
    queries["master_data"] = file.read()
with open('/home/sha/work/yi/data-scripts-queries/queries/query_ecs_yi.txt', "r") as file:
    queries["ecs"] = file.read()
with open('/home/sha/work/yi/data-scripts-queries/queries/query_pwy_yi.txt', "r") as file:
    queries["pwy"] = file.read()
with open('/home/sha/work/yi/data-scripts-queries/queries/query_tax_yi.txt', "r") as file:
    queries["tax"] = file.read()

In [7]:
queries

{'master_data': 'LOAD CSV WITH HEADERS FROM $path AS row\nwith row where row.Age is not null\nMERGE (sub:Subject {name: row.Subject_ID, age: row.Age, sex: row.Sex, race: row.Race})\nWITH sub, row CALL apoc.create.setProperty(sub, row.Visit_ID, row.Visit_number) YIELD node\nMERGE (site:Site {name: row.Sample_body_site})\nMERGE (vis:Visit {visit_number:row.Visit_number, visit_id: row.Visit_ID})\nMERGE (diag:Diagnosis {name:row.Diagnosis})\nMERGE (sub)-[:VISIT]->(vis)\nMERGE (sub)-[:SAMPLED_AT]->(site)\nMERGE (sub)-[:DIAGNOSIS]->(diag)\n',
 'ecs': 'LOAD CSV WITH HEADERS FROM $path AS line\nMATCH (vis:Visit)<-[:VISIT]-(sub:Subject) WHERE vis.visit_id = line.Visit_name\nFOREACH (genus IN (CASE WHEN line.Genus IS NOT NULL THEN [line.Genus] ELSE [] END) |\nMERGE (gen:Genus {name: genus})\nMERGE (enz:Enzyme {name: line.Enzyme})\nMERGE (enz)-[:EXPRESSED_BY {enzabs: line.Abundance_RPKs, visit_id: line.Visit_name, visit_number: vis.visit_number}]->(gen))\nFOREACH (enzyme IN (CASE WHEN line.Genus 

In [8]:
queries_store = []
queries_store.append(
    [
        queries["master_data"],
        "file:///" + 'data/meta/master_data.csv',
        "Importing Master Data",
    ]
)
neo4j_import_folder = '/home/sha/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-c7db64bb-a3cf-43a4-af5c-7438bcb4790d/import/data/csv'
for file_item in files_list:
    print(file_item)
    if os.path.exists(
        str(neo4j_import_folder) + "/" + str(file_item["path"])
    ):
        file_item["files"] = os.listdir(
            neo4j_import_folder + "/" + file_item["path"]
        )

        for file in file_item["files"]:
            queries_store.append(
                [
                    queries[file_item["name"]],
                    f"file:///data/csv/{file_item['path']}/{file}",
                    f"Importing {file_item['name']} data from {file}",
                ]
            )

{'name': 'ecs', 'path': 'ecs', 'files': []}
{'name': 'pwy', 'path': 'pwy', 'files': []}
{'name': 'tax', 'path': 'tax', 'files': []}


In [9]:
# queries_store

In [ ]:
# from neo4j import GraphDatabase
# driver = GraphDatabase.driver(uri, auth=(user, password))
# conn = Neo4jConnection(
#         config.neo4j_config.url, config.neo4j_config.user, config.neo4j_config.pw
#     )
# for query in queries_store:
#     conn.query(query[0], path=query[1])
#     with driver.session() as session:
#         response = list(session.run(query, **kwargs))
#         return response
# driver.close()

In [10]:
from neo4j_connection import Neo4jConnection

In [11]:
NEO4J = {"URL": "bolt://localhost:7687", "USER": "neo4j", "PW": "1qazXSW@"}

neo4j_config_url = NEO4J['URL']
neo4j_config_user = NEO4J['USER']
neo4j_config_pw = NEO4J['PW']
conn = Neo4jConnection(neo4j_config_url, neo4j_config_user, neo4j_config_pw)
for query in queries_store:
    print(f"processing {query[1]}")
    conn.query(query[0], path=query[1])
conn.close()

processing file:///data/meta/master_data.csv
processing file:///data/csv/ecs/ecs_data_110.csv
processing file:///data/csv/ecs/ecs_data_121.csv
processing file:///data/csv/ecs/ecs_data_41.csv
processing file:///data/csv/ecs/ecs_data_128.csv
processing file:///data/csv/ecs/ecs_data_118.csv
processing file:///data/csv/ecs/ecs_data_33.csv
processing file:///data/csv/ecs/ecs_data_150.csv
processing file:///data/csv/ecs/ecs_data_46.csv
processing file:///data/csv/ecs/ecs_data_75.csv
processing file:///data/csv/ecs/ecs_data_168.csv
processing file:///data/csv/ecs/ecs_data_9.csv
processing file:///data/csv/ecs/ecs_data_89.csv
processing file:///data/csv/ecs/ecs_data_23.csv
processing file:///data/csv/ecs/ecs_data_114.csv
processing file:///data/csv/ecs/ecs_data_56.csv
processing file:///data/csv/ecs/ecs_data_54.csv
processing file:///data/csv/ecs/ecs_data_164.csv
processing file:///data/csv/ecs/ecs_data_4.csv
processing file:///data/csv/ecs/ecs_data_13.csv
processing file:///data/csv/ecs/ecs_d